# Vehicle Detection

### Imports

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

%matplotlib inline

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm

## Classifier

### Helper Functions

In [6]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features


# Define a function to return HOG features and visualization
def get_hog_features(img, orient = 9, pix_per_cell = 8, cell_per_block = 2, feature_vec=True):

    features = []
    for channel in range(img.shape[2]):
        features.append(hog(img[:,:,channel], orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
               cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=feature_vec,
               block_norm="L2-Hys"))
    return np.ravel(features)


def calculateFeatures(img):
    
    features = []
    #hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    features.append(bin_spatial(img))
    #features.append(bin_spatial(hsv))
    #features.append(bin_spatial(hls))

    #features.append(color_hist(img))
    #features.append(color_hist(hsv))
    #features.append(color_hist(hls))
    
    features.append(get_hog_features(img))
    #features.append(get_hog_features(hsv))
    #features.append(get_hog_features(hls))
    
    return np.concatenate(features)



### Create Classifier

## Pipeline

### Helper Functions

In [ ]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], window_size = 64, xy_overlap = 0.5):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # number of pixels per step in x/y
    nx_pix_per_step = np.int(window_size * (1 - xy_overlap))
    ny_pix_per_step = np.int(window_size * (1 - xy_overlap))
    # number of windows in x/y
    nx_buffer = np.int(window_size * xy_overlap)
    ny_buffer = np.int(window_size * xy_overlap)
    nx_windows = np.int((xspan - nx_buffer) / nx_pix_per_step) 
    ny_windows = np.int((yspan - ny_buffer) / ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs * nx_pix_per_step + x_start_stop[0]
            endx = startx + window_size
            starty = ys * ny_pix_per_step + y_start_stop[0]
            endy = starty + window_size
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list



### Playground

In [ ]:
vehicles = glob.glob('data/vehicles/*/*.png')
non_vehicles = glob.glob('data/non-vehicles/*/*.png')

car_features = []
non_car_features = []
for car in vehicles:
    img = cv2.imread(car)
    car_features.append(calculateFeatures(img))
for nocar in non_vehicles:
    img = cv2.imread(nocar)
    non_car_features.append(calculateFeatures(img))

# Create an array stack of feature vectors
X = np.vstack((car_features, non_car_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(non_car_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)



#plt.figure(figsize=(20,20))
#plt.subplot(1, 2, 1)
#plt.imshow(car)
#plt.subplot(1, 2, 2)
#plt.imshow(noc)

In [ ]:
clf = svm.SVC()
clf.fit(X_train, y_train)

In [ ]:
score = clf.score(X_test, y_test)
print(score)

In [ ]:
p = 